In [199]:
import pyspark as pys 
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
# #Spark Config (chiamare una volta sola)
conf = SparkConf().setAppName("learning RDD")
#sc = SparkContext(conf=conf) 
spark = SparkSession.builder.getOrCreate()

In [200]:
#Creazione RDD con metodo parallelize
myRDD = sc.parallelize([('Mike', 19), ('June', 18), ('Rachel',16), ('Rob', 18),
('Scott', 17)])
myRDD.take(5)

[('Mike', 19), ('June', 18), ('Rachel', 16), ('Rob', 18), ('Scott', 17)]

In [201]:
#Creazione RDD da file TXT
#/home/lorenzo/Documenti/Tesi/learningPySpark/Chapter03/flight-data
myRDD2 = sc.textFile("./learningPySpark/Chapter03/flight-data/airport-codes-na.txt", minPartitions=1,use_unicode=True).map(lambda element: element.split("\t"))
myRDD2.take(5) #leggo i primi 5 elementi del RDD

myRDD2.getNumPartitions() #leggo il numero di partizioni
myRDD2.count() #Numero di elementi del RDD

527

In [202]:
#Creo RDD airport leggendo il file airport-codes-na.txt
airports = sc.textFile("./learningPySpark/Chapter03/flight-data/airport-codes-na.txt").map(lambda element: element.split("\t"))

#Credo RDD flight leggendo il file departuredelays.csv (splitto con ,)
flight = sc.textFile("./learningPySpark/Chapter03/flight-data/departuredelays.csv").map(lambda element: element.split(","))

In [129]:
#TRASFORMAZIONI



#map(f) passa ogni elemento attraverso f
flight2col=flight.map(lambda c: (c[0],c[1])) #prendo i primi 2 elementi

#filter(f) selezione gli elementi per i quali f ritorna true
airportsWA = airports.filter(lambda c: c[1] == "WA") #filtro gli elementi che hanno la seconda colonna ==WA

#Distinct
airportsD = airports.map(lambda c: c[2]).distinct()

#Sample (campionamento)
airportsS=airports.sample(False,1,1323)

#Repartiton (ripartizione in diverse partizioni)
airports.getNumPartitions() # =2
airportsR = airports.repartition(8)
airportsR.getNumPartitions() #=8

#.zipWithIndex()



8

In [203]:
#Join (separato dagli altri perché dispendioso da eseguire)
flt = flight.map(lambda c: (c[3],c[0])) #c[3] = nome aeroporto
air = airports.map(lambda c: (c[3],c[1]))

flt.join(air).take(5)

[('ABE', ('01011245', 'PA')),
 ('ABE', ('01020600', 'PA')),
 ('ABE', ('01021245', 'PA')),
 ('ABE', ('01020605', 'PA')),
 ('ABE', ('01031245', 'PA'))]

In [148]:
#reduceByKey(f) + pipelining
#Raggruppo sul campo chiave (aereoporto di partenza) sommando il ritardo
myList=(
    flight
    .zipWithIndex()
    .filter(lambda c: c[1]>0) #elimino intestazione
    .map(lambda c: c[0]) #elimino indice
    #c[3] = aeroporto di partenza, c[1] = ritardo
    .map(lambda c: (c[3], int(c[1]))) #CAST A INTERO PER SOMMA
    .reduceByKey(lambda x,y:x+y)
    .take(5)
)
print(myList)
#print(myList[0][1])

[('ABE', 5113), ('ACT', 392), ('ACV', 8429), ('ADQ', -254), ('AEX', 10193)]


In [189]:
#AZIONI
#lettura dataset
flight.take(5) #ritorn i primi 5 elementi
#flight.collect() #ritorna tutto il dataset (meglio non usare)

#operazioni di aggregazione
#Filtro voli da SEA a SFO
#Leggo il valore facendo il casting a INT
#Ottengo la somma 
flight\
.filter(lambda c: c[3]=="SEA" and c[4]=="SFO")\
.map(lambda c: int(c[1]))\
.reduce(lambda x,y: x+y)

flight.count()



1391579

In [194]:
#saveAsTextFile
#un file per ogni partizione
airports.getNumPartitions() # = 2
airports.saveAsTextFile("/tmp/airports")